In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, StandardScaler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession
from pyspark.sql.functions import expm1
from pyspark.sql.functions import log1p
from pyspark.sql.functions import monotonically_increasing_id
from xgboost.spark import SparkXGBRegressor

In [5]:
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "15g")
    .config("spark.sql.parquet.enableVectorizedReader", "false")
    .getOrCreate()
)

In [6]:
data = spark.read.parquet('../data/curated/')
data.columns

['hourly_timestamp',
 'PULocationID',
 'pickup_hour_of_day',
 'pickup_day_of_week',
 'pickup_month',
 'pickup_borough',
 'pickup_at_airport',
 'num_trips',
 'pickup_num_businesses',
 'temperature_2m',
 'relative_humidity_2m',
 'rain',
 'snowfall',
 'wind_speed_10m']

Since the number of trips is a highly skewed count variable, we will use the log of the number of trips as the target variable. We will also add 1 to the number of trips before taking the log to avoid taking the log of 0.

In [7]:
data = data.withColumn("log_num_trips", log1p("num_trips"))

In [8]:
train_data = data.filter(data['pickup_month'] <= 5)  # Months 1-5 for training
test_data = data.filter(data['pickup_month'] > 5)  # Month 6 for testing

In [9]:
# Index and One-Hot Encode categorical feature
indexers = [
    StringIndexer(inputCol=column, outputCol=f"{column}_index").setHandleInvalid("keep")
    for column in ['pickup_day_of_week']
]

encoders = [
    OneHotEncoder(inputCol=f"{column}_index", outputCol=f"{column}_ohe").setHandleInvalid("keep")
    for column in ['pickup_day_of_week']
]

In [10]:
assembler = VectorAssembler(
    inputCols=[
        'pickup_hour_of_day',
        "pickup_at_airport",
        'pickup_num_businesses',
        'temperature_2m',
        'relative_humidity_2m',
        'rain',
        'snowfall',
        'wind_speed_10m',
        'pickup_day_of_week_ohe',
    ],
    outputCol="features"
).setHandleInvalid("keep")

In [11]:
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

## Linear Regression
with Lasso Regularization

In [12]:
lr = LinearRegression(featuresCol='scaled_features', labelCol='log_num_trips', elasticNetParam=1)

In [13]:
pipeline_lr = Pipeline(stages=indexers + encoders + [assembler, scaler, lr])

In [14]:
model = pipeline_lr.fit(train_data)

24/08/24 15:38:35 WARN Instrumentation: [122499c5] regParam is zero, which might cause numerical instability and overfitting.
24/08/24 15:38:37 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/08/24 15:38:37 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
24/08/24 15:38:37 WARN Instrumentation: [122499c5] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [15]:
# Print the columns and their corresponding coefficients
model.stages[-1].coefficients

DenseVector([0.3516, 0.2006, 0.5932, 0.0158, 0.0132, -0.0042, -0.0203, -0.0113, 0.0106, -0.0352, -0.038, 0.0495, 0.0398, -0.0022, -0.0242, 0.0])

In [16]:
predictions = model.transform(test_data)
predictions = predictions.withColumn("lr_prediction", expm1(predictions["prediction"]))

In [17]:
evaluator = RegressionEvaluator(
    labelCol="num_trips", predictionCol="lr_prediction", metricName="rmse")

rmse = evaluator.evaluate(predictions)
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print(f"Root Mean Squared Error (RMSE) on test data = {rmse}")
print(f"R2 on test data = {r2}")

Root Mean Squared Error (RMSE) on test data = 108.68464173837049
R2 on test data = 0.0267485443305312


In [18]:
errors = predictions.select('pickup_hour_of_day', 'pickup_day_of_week', 'pickup_month', 'num_trips', 'lr_prediction',
                            'pickup_num_businesses', "PULocationID")

## XGBoost

In [19]:
xgb_regressor = SparkXGBRegressor(
    features_col="scaled_features",
    label_col="log_num_trips",
    num_workers=2,
)

In [20]:
pipeline_xgb = Pipeline(stages=indexers + encoders + [assembler, scaler, xgb_regressor])

In [21]:
model = pipeline_xgb.fit(train_data)

2024-08-24 15:38:43,885 INFO XGBoost-PySpark: _fit Running xgboost-2.1.1 on 2 workers with
	booster params: {'objective': 'reg:squarederror', 'device': 'cpu', 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-08-24 15:38:49,356 INFO XGBoost-PySpark: _train_booster Training on CPUs 2]
[15:38:50] Task 0 got rank 0
[15:38:50] Task 1 got rank 1
2024-08-24 15:38:54,031 INFO XGBoost-PySpark: _fit Finished xgboost training!   


In [22]:
predictions = model.transform(test_data)
predictions = predictions.withColumn("xgb_prediction", expm1(predictions["prediction"]))

In [23]:
evaluator = RegressionEvaluator(
    labelCol="num_trips", predictionCol="xgb_prediction", metricName="rmse")

rmse = evaluator.evaluate(predictions)
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print(f"Root Mean Squared Error (RMSE) on test data = {rmse}")
print(f"R2 on test data = {r2}")

2024-08-24 15:38:54,626 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2024-08-24 15:38:57,354 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


Root Mean Squared Error (RMSE) on test data = 45.139417409822194
R2 on test data = 0.8321193643806969


In [24]:
# Add XGBoost predictions to errors dataframe based on index
errors = errors.withColumn("index", monotonically_increasing_id())
predictions = predictions.withColumn("index", monotonically_increasing_id())
errors = errors.join(predictions.select("index", "xgb_prediction"), "index", "left").drop("index")

## Decision Tree Regressor

In [25]:
decision_tree_regressor = DecisionTreeRegressor(
    featuresCol="scaled_features",
    labelCol="log_num_trips"
)

In [26]:
pipeline_dt = Pipeline(stages=indexers + encoders + [assembler, scaler, decision_tree_regressor])

In [27]:
model = pipeline_dt.fit(train_data)

In [28]:
predictions = model.transform(test_data)
predictions = predictions.withColumn("dt_prediction", expm1(predictions["prediction"]))

In [29]:
evaluator = RegressionEvaluator(
    labelCol="num_trips", predictionCol="dt_prediction", metricName="rmse")

rmse = evaluator.evaluate(predictions)
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print(f"Root Mean Squared Error (RMSE) on test data = {rmse}")
print(f"R2 on test data = {r2}")

Root Mean Squared Error (RMSE) on test data = 92.0683461386725
R2 on test data = 0.30159170423746706


In [30]:
# Add Decision Tree Regressor predictions to errors dataframe based on index
errors = errors.withColumn("index", monotonically_increasing_id())
predictions = predictions.withColumn("index", monotonically_increasing_id())
errors = errors.join(predictions.select("index", "dt_prediction"), "index", "left").drop("index")

In [31]:
# Write errors dataframe to parquet for further analysis
errors.write.mode('overwrite').parquet('../data/errors/')

2024-08-24 15:39:11,162 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
